In [ ]:
import pandas as pd
import sqlite3
import plotly.graph_objects as go
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import numpy as np
import plotly.express as px




# Connect to the SQLite database
db_path = "database.sqlite"
conn = sqlite3.connect(db_path)

# Inspect table relationships
def fetch_data(query):
    return pd.read_sql_query(query, conn)

player_attributes = fetch_data("SELECT * FROM Player_Attributes")
team_attributes = fetch_data("SELECT * FROM Team_Attributes")
match_data = fetch_data("SELECT * FROM Match")
player_data = fetch_data("SELECT * FROM Player")
league_data = fetch_data("SELECT * FROM League")
country_data = fetch_data("SELECT * FROM Country")
team_data = fetch_data("SELECT * FROM Team")

# Display table relationships
print("Player Attributes Sample:", player_attributes.head())
print("Team Attributes Sample:", team_attributes.head())
print("Match Data Sample:", match_data.head())
print("Player Data Sample:", player_data.head())
print("League Data Sample:", league_data.head())
print("Country Data Sample:", country_data.head())
print("Team Data Sample:", team_data.head())

In [ ]:
# Merge player and player attributes
df_players = player_attributes.merge(player_data, on="player_api_id", how="inner")

# Merge match data with league and country context
league_data = league_data.rename(columns={'id': 'league_id', 'name': 'league_name'})
country_data = country_data.rename(columns={'id': 'country_id', 'name': 'country_name'})

match_data = match_data.merge(league_data, on="league_id", how="left")
match_data = match_data.merge(country_data, left_on="league_id", right_on="country_id", how="left")

# Merge match data with team information
team_data = team_data.rename(columns={"team_api_id": "team_id", "team_long_name": "team_name"})
match_data = match_data.merge(team_data, left_on="home_team_api_id", right_on="team_id", how="left")
match_data.rename(columns={"team_name": "home_team_name"}, inplace=True)
match_data = match_data.merge(team_data, left_on="away_team_api_id", right_on="team_id", how="left")
match_data.rename(columns={"team_name": "away_team_name"}, inplace=True)

# Display consolidated data
print("Consolidated Player Data:", df_players.head())
print("Consolidated Match Data:", match_data.head())

In [ ]:

# Impute missing values in critical columns for players
imputer = SimpleImputer(strategy="mean")
columns_to_impute = ["overall_rating", "potential", "height", "weight"]
df_players[columns_to_impute] = imputer.fit_transform(df_players[columns_to_impute])

# Impute missing values for team attributes
team_imputer = SimpleImputer(strategy="most_frequent")
team_attributes = team_attributes.fillna(method="ffill")
team_attributes = team_attributes[["team_fifa_api_id", "buildUpPlaySpeed", "chanceCreationPassing", "defenceAggression"]]

# Impute missing values for match data
match_imputer = SimpleImputer(strategy="mean")
match_data["home_team_goal"] = match_imputer.fit_transform(match_data[["home_team_goal"]])
match_data["away_team_goal"] = match_imputer.fit_transform(match_data[["away_team_goal"]])

# Standardize formats
columns_to_normalize = ["overall_rating", "potential", "height", "weight"]
df_players[columns_to_normalize] = df_players[columns_to_normalize].apply(lambda x: (x - x.mean()) / x.std())

In [ ]:
# Check if 'season' exists, and create it if necessary
if "season" not in df_players.columns:
    if "date" in df_players.columns:
        df_players["season"] = pd.to_datetime(df_players["date"]).dt.year
    else:
        raise KeyError("The 'season' column is missing, and no date column is available to derive it.")

# Define a performance metric
df_players["performance_score"] = df_players["overall_rating"] * 0.5 + df_players["potential"] * 0.5

# Extract top 5 best performers by season
best_players = (
    df_players.groupby(["season", "player_name"], as_index=False)["performance_score"]
    .max()
    .sort_values(["season", "performance_score"], ascending=[True, False])
)

# Keep only the top 5 players per season
top_players_per_season = best_players.groupby("season").head(5)

# Initialize the figure
fig = go.Figure()

# Loop through each season and plot the data
for season in top_players_per_season["season"].unique():
    season_data = top_players_per_season[top_players_per_season["season"] == season]
    fig.add_trace(
        go.Bar(
            x=season_data["player_name"],
            y=season_data["performance_score"],
            name=str(season),
        )
    )

# Update layout for readability
fig.update_layout(
    title="Top 5 Performers by Season",
    xaxis_title="Players",
    yaxis_title="Performance Score",
    barmode="group",
    xaxis_tickangle=-45,
    height=500,
    width=1000
)
fig.update_layout(
template = 'plotly_dark')
fig.show()


In [ ]:

# Step 1: Drop NaNs from performance_score
valid_players = df_players.dropna(subset=["performance_score"])

# Step 2: Filter players with at least 2 records
players_with_multiple_scores = valid_players.groupby("player_name").filter(lambda x: len(x) > 1)

# Step 3: Calculate Mean Absolute Deviation (MAD) and scale it
consistency = (
    players_with_multiple_scores.groupby("player_name")["performance_score"]
    .apply(lambda x: (x - x.mean()).abs().mean())
    .sort_values()
)

# Step 4: Filter players with meaningful MAD (> small threshold)
consistency_filtered = consistency[consistency > 0.01]  # Threshold to exclude near-zero values

# Step 5: Scale MAD values for better visualization (optional step)
consistency_scaled = consistency_filtered * 100  # Scale MAD for clarity

# Step 6: Select top 10 most consistent players
consistent_players = consistency_scaled.head(10)

# Debug Output
print("Top 10 Most Consistent Players with MAD:")
print(consistent_players)

# Step 7: Plot the graph
fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=consistent_players.index,
        y=consistent_players.values,
        marker_color="springgreen"
    )
)
fig.update_layout(
    title="Most Consistent Players (Based on Mean Absolute Deviation)",
    xaxis_title="Players",
    yaxis_title="Consistency (Low MAD, Scaled)",
    xaxis_tickangle=-45,
    height=500,
    width=1000,
)
fig.update_layout(
template = 'plotly_dark')
fig.show()


In [ ]:
# Calculate wins, losses, and draws for each team
match_data["result"] = match_data["home_team_goal"] - match_data["away_team_goal"]
match_data["home_win"] = match_data["result"] > 0
match_data["away_win"] = match_data["result"] < 0

dominating_teams = (
    match_data.groupby("home_team_name")["home_win"].sum() +
    match_data.groupby("away_team_name")["away_win"].sum()
).sort_values(ascending=False)
fig = go.Figure()
fig.add_trace(go.Bar(x=dominating_teams.index[:10], y=dominating_teams.values[:10]))
fig.update_layout(title="Top Dominating Teams", xaxis_title="Teams", yaxis_title="Wins")
fig.update_layout(
template = 'plotly_dark')
fig.show()

In [ ]:

# Filter out non-numeric ID columns
numeric_team_attributes = team_attributes.select_dtypes(include=["number"]).drop(columns=["team_fifa_api_id"], errors="ignore")

# Calculate descriptive statistics
team_metrics = numeric_team_attributes.describe()

# Plot the updated team metrics
fig = go.Figure()
fig.add_trace(go.Bar(x=team_metrics.columns, y=team_metrics.loc["mean"], name="Mean", marker_color="yellow"))
fig.add_trace(go.Bar(x=team_metrics.columns, y=team_metrics.loc["std"], name="Std Dev", marker_color="tomato"))

fig.update_layout(
    title="Team Metrics Summary (Excluding ID Columns)",
    barmode="group",
    xaxis_title="Metrics",
    yaxis_title="Values",
    height=500,
    width=1000
)
fig.update_layout(
template = 'plotly_dark')
fig.show()

In [ ]:

# Calculate total goals for each match
match_data["total_goals"] = match_data["home_team_goal"] + match_data["away_team_goal"]

# Sort matches by total goals and select the top 10
high_scoring_matches = match_data.sort_values(by="total_goals", ascending=False).head(10)

# Plot high-scoring matches
fig = go.Figure()
fig.add_trace(go.Bar(
    x=high_scoring_matches["home_team_name"] + " vs " + high_scoring_matches["away_team_name"],
    y=high_scoring_matches["total_goals"],
    name="Total Goals",
    marker_color="fuchsia"
))

# Update layout
fig.update_layout(
    title="Top 10 High Scoring Matches",
    xaxis_title="Match (Home vs Away)",
    yaxis_title="Total Goals",
    xaxis_tickangle=-45,
    height=500,
    width=1000
)
fig.update_layout(
template = 'plotly_dark')

fig.show()

#### **Compare Home and Away Performance**

In [ ]:
home_performance = match_data.groupby("home_team_name")["home_team_goal"].mean()
away_performance = match_data.groupby("away_team_name")["away_team_goal"].mean()
fig = go.Figure()
fig.add_trace(go.Bar(x=home_performance.index[:10], y=home_performance.values[:10], name="Home Performance"))
fig.add_trace(go.Bar(x=away_performance.index[:10], y=away_performance.values[:10], name="Away Performance"))
fig.update_layout(title="Home vs Away Performance", barmode="group", xaxis_title="Teams", yaxis_title="Average Goals", template = 'plotly_dark')
fig.show()

### **Trend Analysis**

#### **Player and Team Performance Trends**

In [ ]:

# Step 1: Group player performance trends
player_trends = df_players.groupby("season")["performance_score"].mean()
player_trends.index = player_trends.index.astype(str)  # Convert to string

# Step 2: Group team performance trends
team_trends = (
    match_data.groupby("season")[["home_team_goal", "away_team_goal"]]
    .mean()
    .sum(axis=1)  # Sum of average home and away goals
)
team_trends.index = team_trends.index.astype(str)  # Convert to string

# Step 3: Align the indices of both trends
all_seasons = sorted(set(player_trends.index).union(set(team_trends.index)))
player_trends = player_trends.reindex(all_seasons, fill_value=0)
team_trends = team_trends.reindex(all_seasons, fill_value=0)

# Debug Output
print("Aligned Player Trends:")
print(player_trends)

print("\nAligned Team Trends:")
print(team_trends)

# Step 4: Plot performance trends over seasons
fig = go.Figure()

# Player Performance Trend
fig.add_trace(
    go.Scatter(
        x=player_trends.index,
        y=player_trends.values,
        mode="lines+markers",
        name="Player Performance",
        line=dict(color="white"),
    )
)

# Team Performance Trend
fig.add_trace(
    go.Scatter(
        x=team_trends.index,
        y=team_trends.values,
        mode="lines+markers",
        name="Team Performance",
        line=dict(color="red"),
    )
)

# Update layout
fig.update_layout(
    title="Performance Trends Over Seasons",
    xaxis_title="Seasons",
    yaxis_title="Average Performance / Goals",
    xaxis_tickangle=-45,
    height=500,
    width=1000,
)
fig.update_layout(
template = 'plotly_dark')
fig.show()


## **3. Clustering**

### **Objective**
Use clustering techniques to group players and teams based on attributes and styles of play. This will help identify groups with similar characteristics, aiding strategic analysis.

### **Player Clustering**

#### **Clustering Criteria**

In [ ]:


# Select player attributes for clustering
player_features = df_players[["overall_rating", "potential", "height", "weight"]]
scaler = StandardScaler()
player_features_scaled = scaler.fit_transform(player_features)

# Apply K-Means clustering
kmeans = KMeans(n_clusters=3, random_state=42)
df_players["cluster"] = kmeans.fit_predict(player_features_scaled)

# Visualize clustering results
fig = px.scatter(df_players, x="overall_rating", y="potential", color="cluster", 
                 hover_data=["player_name"], title="Player Clustering")
fig.update_layout(
template = 'plotly_dark')
fig.show()

#### **Cluster Interpretation**
- **Prolific Attackers:** High rating and offensive potential.
- **Robust Defenders:** Large size, heavy weight, high defensive ratings.
- **Balanced Midfielders:** Well-distributed overall and potential scores.


### **Team Clustering**

#### **Clustering Criteria**

In [ ]:
# Select team attributes for clustering
team_features = team_attributes[["buildUpPlaySpeed", "chanceCreationPassing", "defenceAggression"]]
team_features_scaled = scaler.fit_transform(team_features)

# Apply K-Means clustering
team_kmeans = KMeans(n_clusters=3, random_state=42)
team_attributes["cluster"] = team_kmeans.fit_predict(team_features_scaled)

# Visualize clustering results
fig = px.scatter_3d(team_attributes, x="buildUpPlaySpeed", y="chanceCreationPassing", z="defenceAggression", 
                    color="cluster", title="Team Clustering")

# Adjust the size of the plot
fig.update_layout(
    width=1000,  # Set the width of the plot
    height=600,  # Set the height of the plot
    template="plotly_dark"  # Apply dark theme
)

fig.show()

#### **Cluster Interpretation**
- **Offensive Teams:** High build speed, many goals scored.
- **Defensive Teams:** High defensive pressure, few goals conceded.
- **Balanced Teams:** Moderate metrics, balanced style of play.


## **4. Clustering Techniques**

### **K-Means-Based Approach**
- **Advantages:** Simple and efficient for well-separated clusters.
- **Application:** Segment players based on physical and performance attributes.

### **DBSCAN-Based Approach**
- **Advantages:** Ideal for identifying atypical players or teams.
- **Application:** Detect exceptional talent or unique team styles.


## **5. Presentation of Results**

### **Summary Tables**

In [ ]:
# Rankings of the best players, teams, and matches
best_players = df_players.nlargest(30, "performance_score")[["player_name", "performance_score"]]
print("Top 10 Players:")
best_players.head(30)

In [ ]:
dominating_teams = (
    match_data.groupby("home_team_name")["home_win"].sum() +
    match_data.groupby("away_team_name")["away_win"].sum()
).sort_values(ascending=False)
dominating_teams.head(10)

In [ ]:
high_scoring_matches = match_data.nlargest(10, ["home_team_goal", "away_team_goal"])
print("Top 10 High Scoring Matches:")
high_scoring_matches.head()

### **Graphs and Visualizations**

#### **Team Performance Comparisons**

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=dominating_teams.index, y=dominating_teams.values))
fig.update_layout(title="Top Dominating Teams", xaxis_title="Teams", yaxis_title="Wins", template='plotly_dark')
fig.show()

#### **Player or Team Clusters**

In [ ]:
fig = px.scatter(df_players, x="overall_rating", y="potential", color="cluster",  color_continuous_scale="Inferno",  # Gradient scale: 'Viridis', 'Bluered', 'Inferno', etc.

                 hover_data=["player_name"], title="Player Clustering")
fig.update_layout(template='plotly_dark')

fig.show()

#### **Seasonal Trends**

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=player_trends.index, y=player_trends.values, mode="lines+markers", name="Player Performance"))
fig.add_trace(go.Scatter(x=team_trends.index, y=team_trends.values, mode="lines+markers", name="Team Performance"))
fig.update_layout(title="Performance Trends Over Seasons", xaxis_title="Seasons", yaxis_title="Average Performance", template="plotly_dark")
fig.show()

### **Key Interpretations**
- The clustering results highlight distinct groups of players and teams, such as prolific attackers, robust defenders, and balanced midfielders.
- Dominating teams showcase consistently high performance, revealing opportunities for benchmarking.
- Seasonal trends provide insight into performance evolution, which can guide training strategies.

### **Recommendations**
- Focus on nurturing balanced players to maintain versatility across positions.
- Prioritize defensive strategies for teams lagging in defensive metrics.
- Use clustering insights to tailor coaching and training for specific player groups.
